# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [4]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.4 MB/s eta 0:00:0000:0100:02
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 1.5 MB/s eta 0:00:00a 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840633 sha256=c30393c78cffefc78c39ddfd4cbc827409312574d9e5852cfaff872fd5b75436
  Stored in directory: /home/jovyan/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [1]:
!pwd

/home/jovyan/work


In [2]:
! hadoop fs -mkdir /data/
! hadoop fs -copyFromLocal clickstream.csv /data/
! hadoop fs -ls -h /data/

mkdir: `/data': File exists
copyFromLocal: `/data/clickstream.csv': File exists
Found 1 items
-rw-r--r--   1 jovyan supergroup     30.7 M 2024-10-23 20:13 /data/clickstream.csv


In [2]:
import json
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import functions as F
from pyspark.sql import SparkSession
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-10-27 10:16:09,963 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [7]:
df = se.read.format("csv").option("header", "true").option("sep", "\t").load("hdfs:/data/clickstream.csv")
df.show(5)

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
+-------+----------+------------+----------+----------+
only showing top 5 rows



In [8]:
df.registerTempTable("click")

routes_df = se.sql("""
SELECT user_id, session_id, COUNT(*) AS cnt, 
       CONCAT_WS('-', COLLECT_LIST(event_page)) AS route
FROM (
    SELECT user_id, session_id, event_page
    FROM (SELECT user_id, session_id, event_page, no
FROM (
    SELECT user_id, session_id, event_page, no,
           LAG(event_page, 1, NULL) OVER (PARTITION BY user_id, session_id ORDER BY no) AS prev_event
    FROM 
    (SELECT user_id, session_id, event_page, 
       ROW_NUMBER() OVER (PARTITION BY user_id, session_id ORDER BY timestamp ASC) AS no
FROM (
    SELECT *, 
           MAX(CASE WHEN event_type NOT IN ('page', 'event') THEN 1 ELSE 0 END) OVER (PARTITION BY user_id, session_id ORDER BY timestamp ASC) AS error_tag
    FROM click
) filtered
WHERE error_tag = 0
ORDER BY user_id, session_id, no) as grp
) filtered
WHERE event_page != prev_event OR prev_event IS NULL
ORDER BY user_id, session_id, no) as wo_dup
    ORDER BY user_id, session_id, no
) sorted_events
GROUP BY user_id, session_id
""")

routes_df.show(20)

+-------+----------+---+--------------------+
|user_id|session_id|cnt|               route|
+-------+----------+---+--------------------+
|      0|       874|  3|  main-rabota-online|
|      0|       888|  1|                main|
|      0|       907|  6|main-digital-news...|
|      1|      1013|  4|main-tariffs-onli...|
|      1|      1026|  1|                main|
|      1|      1030|  2|        main-archive|
|      1|      1036|  1|                main|
|      1|      1038|  1|                main|
|      1|       963|  2|           main-news|
|      1|       979|  5|main-rabota-archi...|
|      1|       992|  5|main-rabota-inter...|
|     10|       730| 10|main-archive-tari...|
|     10|       753|  2|         main-rabota|
|     10|       762| 12|main-internet-arc...|
|    100|       937| 15|main-archive-onli...|
|    100|       939|  3|  main-rabota-online|
|    100|       984|  6|main-internet-tar...|
|   1000|        86|  1|                main|
|   1000|        91|  2|        ma

In [9]:
routes_rdd = routes_df.rdd.map(lambda row: (row['route'], 1)) \
                          .reduceByKey(lambda a, b: a + b) \
                          .sortBy(lambda x: x[1], ascending=False)

top_routes = routes_rdd.take(10)

In [10]:
top_30 = routes_rdd.take(30)
top_30_tsv = [ x[0]+"\t"+str(x[1]) for x in top_30]
output_path = "top_30_routes.tsv"
f = open(output_path, "w")
for line in top_30_tsv:
    f.write(line+"\n")
f.close()

In [11]:
result_dict = {route: count for route, count in top_routes}
output_path = "result.json"
with open(output_path, "w") as f:
    json.dump(result_dict, f)

print(result_dict)
print(f"Top 10 routes saved to {output_path}")

{'main': 8184, 'main-archive': 1113, 'main-rabota': 1047, 'main-internet': 897, 'main-bonus': 870, 'main-news': 769, 'main-tariffs': 677, 'main-online': 587, 'main-vklad': 518, 'main-rabota-archive': 170}
Top 10 routes saved to result.json
